# Competición CNN 

Autores: David García Lleyda, Álvaro Martínez Parpolowicz, Alexis Gómez Chimeno

Clase: MAIS 5ºA

## 1. Librerías

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.layers import RandomRotation
from tensorflow.keras.models import Model
import requests
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
import os
from tensorflow.image import stateless_random_flip_up_down
import tensorflow as tf
import json
import cv2
import os
import shutil
 

## 2. Data augmentation

A partir de aquí se implementa el código con el que se ha aumentado el dataset de train mediante técnicas de data augmentation.

Se ha preferido generar las imágenes y guardarlas que implementarlas a través del generador. De esta manera, ahorramos tiempo cargando una y otra vez las imágenes que debe modificar el generador para dar el conjunto final.

Aunque solo se deba generar nuevas imágenes modificadas con respecto a los datos de train, se deja comentado el código necesario para generar respecto a cada una de las carpetas restantes (validación y test).

In [2]:
def load_image(file):
    img = cv2.imread(file)
    return img


######################################################################
relative_path_train='./Sports/train/'
# relative_path_valid='./Sports/valid/'
# relative_path_test='./Sports/test/'
######################################################################


######################################################################
with open('one_hot_train_by_folders.json','r') as file:
    train_data = json.load(file)
    file.close()

# with open('one_hot_valid_by_folders.json','r') as file:
#     valid_data = json.load(file)
#     file.close()

# with open('one_hot_test_by_folders.json','r') as file:
#     test_data = json.load(file)
#     file.close()
######################################################################


######################################################################
# imgs_todo_aug_test = []
# for i in test_data:
#     imgs_todo_aug_test.append(load_image(relative_path_test + i))
    
imgs_todo_aug_train = []
for i in train_data:
    imgs_todo_aug_train.append(load_image(relative_path_train + i))

# imgs_todo_aug_valid = []
# for i in valid_data:
#     imgs_todo_aug_valid.append(load_image(relative_path_valid + i))
######################################################################


In [3]:
output_dirname = 'Sports_augmented'
if not os.path.exists(output_dirname):
    os.makedirs(output_dirname)

######################################################################
# test_dirname = "test"
# output_test_dirname = output_dirname+"/"+test_dirname
# if not os.path.exists(output_test_dirname):
#     os.makedirs(output_test_dirname)

train_dirname = "train"
output_train_dirname = output_dirname+"/"+train_dirname
if not os.path.exists(output_train_dirname):
    os.makedirs(output_train_dirname)

# valid_dirname = "valid"
# output_valid_dirname = output_dirname+"/"+valid_dirname
# if not os.path.exists(output_valid_dirname):
#     os.makedirs(output_valid_dirname)
######################################################################


######################################################################
# for i in os.listdir(relative_path_test):
    
#     if not os.path.exists(output_test_dirname+"/"+i):
#         os.makedirs(output_test_dirname+"/"+i)
    
for i in os.listdir(relative_path_train):
    if not os.path.exists(output_train_dirname+"/"+i):
        os.makedirs(output_train_dirname+"/"+i)
    
# for i in os.listdir(relative_path_valid):
#     if not os.path.exists(output_valid_dirname+"/"+i):
#         os.makedirs(output_valid_dirname+"/"+i)
######################################################################

In [8]:
augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.2),  # rotation in degrees
    # tf.keras.layers.RandomZoom(0.2),  # random zoom
    tf.keras.layers.RandomFlip("horizontal")  # horizontal flip
])

######################################################################
# aug_test = list(map(lambda x: augmentation(x), imgs_todo_aug_test))

aug_train = []

for i,j in zip(imgs_todo_aug_train, train_data):
    # aug_train.append(augmentation(i))
    aug_image = augmentation(i)
    folder, image_name = j.split("/")
    cv2.imwrite(output_train_dirname+"/"+folder+"/augmented_"+image_name, aug_image.numpy())

# aug_train = list(map(lambda x: augmentation(x), imgs_todo_aug_train))

# aug_valid = list(map(lambda x: augmentation(x), imgs_todo_aug_valid))
######################################################################

Como el código genera una nueva carpeta con las imágenes modificadas, copiamos tanto las imágenes desde la carpeta original de train dada y la carpeta generada a una carpeta final.

In [ ]:
source = './Sports/train/'
source_augmented = './Sports_augmented/train/'
destination = './Sports_Final/train/'
# create destination folder if not exists
sports_final= './Sports_Final/'

if not os.path.exists(sports_final):
	os.mkdir(sports_final)
if not os.path.exists(destination):
	os.mkdir(destination)

In [16]:
def copy_images(src_folder, dest_folder):
    for root, _, files in os.walk(src_folder):
        for file in files:
            src_path = os.path.join(root, file)
            dest_path = os.path.join(dest_folder, os.path.relpath(src_path, src_folder))
 
            # Create the destination directory if it doesn't exist
            os.makedirs(os.path.dirname(dest_path), exist_ok=True)
 
            # Copy the file to the destination
            shutil.copy(src_path, dest_path)
 
def merge_train_folders(src_folder1, src_folder2, dest_folder):
    for sport_folder1 in os.listdir(src_folder1):
        sport_src_folder1 = os.path.join(src_folder1, sport_folder1)
        sport_src_folder2 = os.path.join(src_folder2, sport_folder1)
        sport_dest_folder = os.path.join(dest_folder, sport_folder1)
 
        os.makedirs(sport_dest_folder, exist_ok=True)
 
        # Copy images from the first train folder
        copy_images(sport_src_folder1, sport_dest_folder)
 
        # Copy images from the second train folder
        copy_images(sport_src_folder2, sport_dest_folder)
 
# Specify the source and destination folders
src_folder1 = "Sports/train"
src_folder2 = "Sports_augmented/train"
dest_folder = "Sports_final/train"
 
# Merge the images from both train folders
merge_train_folders(src_folder1, src_folder2, dest_folder)
 
print("Images merged successfully.")

Images merged successfully.
